In [1]:
import requests
import re
import os
import pandas as pd
import numpy as np
import sqlite3
import datetime
import cv2
from sqlalchemy import create_engine

In [2]:
## Download urls function
def down_urls(html):
    url_pics = re.findall('<img alt=".*?" src="(.*?)" height="', str(html.text), re.S)
    return url_pics

## Download image function
def down_pics(url_list,address):
    i=0
    name_list = []
    local_list = []
    down_time_list = []
    width_list = []
    height_list = []
    color_list = []
    for each in url_list:
        print('Downloading the' + str(i) + '，image address：' + str(each))
        try:
            pic = requests.get(each, timeout=10)
        except requests.exceptions.ConnectionError:
            print('Error！This image cannot be downloaded')
            continue
            
        # create local path
        dir = address  + str(i) + '.jpg'
        
        # record image name, local location, download time
        name_list.append(i)
        local_list.append(dir)
        down_time_list.append(datetime.datetime.now())
        with open(dir, 'wb') as file:
            file.write(pic.content)
            
        # read the image width, height and channel
        img = cv2.imread(dir)
        sp = img.shape
        height_list.append(sp[0])#height(rows) of image
        width_list.append(sp[1])#width(colums) of image
        color_list.append(sp[2])#the pixels value is made up of three primary colors
        
        i+=1
        
    ## create and save image info table
    df1 = pd.DataFrame({'Image_name':name_list,'Image_url':url_list,\
                        'Width':width_list,'Height':height_list,'Color_channels':color_list,\
                  'Download_datetime':down_time_list,'Image_location':local_list})
    return df1
    
            
## Select data from database function
def select(query):
    conn = sqlite3.connect('foo.db')
    cursor = conn.cursor()
    cursor.execute(query)
    values = cursor.fetchall()
    cursor.close()
    conn.close()
    return values

In [3]:
## 1. Download data and load them into sqlite db
################################################
# Eg. 100+ images from Amazon 100 Best sellers in women clothings page
url1 = 'https://www.amazon.com/Best-Sellers-Womens-Clothing/zgbs/fashion/1040660/ref=zg_bs_nav_2_7147440011'
url2 = 'https://www.amazon.com/Best-Sellers-Womens-Clothing/zgbs/fashion/1040660/ref=zg_bs_pg_2?_encoding=UTF8&pg=2'
result1 = requests.get(url1)
url_list = down_urls(result1)
result2 = requests.get(url2)
url_list.extend(down_urls(result2))
df_1 = pd.DataFrame({'urls':url_list})

# load pandas df into sql database
engine = create_engine('sqlite:///foo.db')
print(df_1)
df_1.to_sql('image_urls', engine, if_exists='replace')

                                                  urls
0    https://images-na.ssl-images-amazon.com/images...
1    https://images-na.ssl-images-amazon.com/images...
2    https://images-na.ssl-images-amazon.com/images...
3    https://images-na.ssl-images-amazon.com/images...
4    https://images-na.ssl-images-amazon.com/images...
..                                                 ...
113  https://images-na.ssl-images-amazon.com/images...
114  https://images-na.ssl-images-amazon.com/images...
115  https://images-na.ssl-images-amazon.com/images...
116  https://images-na.ssl-images-amazon.com/images...
117  https://images-na.ssl-images-amazon.com/images...

[118 rows x 1 columns]


In [4]:
## 2. download images into local address & 3. Save these images structured in a tabular format
########################################
urls_list = [url[0] for url in select('select urls from image_urls')]
address = '/Users/jamesyang/Desktop/Technical Challenge/download_images/'
df_2 = down_pics(urls_list,address)

# load pandas df into sql database
engine = create_engine('sqlite:///foo.db')
df_2.to_sql('image_info', engine, if_exists='replace')

In [8]:
## Table 1: 
print(df_1)

## select from sqlite db
select('select * from image_urls')

                                                  urls
0    https://images-na.ssl-images-amazon.com/images...
1    https://images-na.ssl-images-amazon.com/images...
2    https://images-na.ssl-images-amazon.com/images...
3    https://images-na.ssl-images-amazon.com/images...
4    https://images-na.ssl-images-amazon.com/images...
..                                                 ...
113  https://images-na.ssl-images-amazon.com/images...
114  https://images-na.ssl-images-amazon.com/images...
115  https://images-na.ssl-images-amazon.com/images...
116  https://images-na.ssl-images-amazon.com/images...
117  https://images-na.ssl-images-amazon.com/images...

[118 rows x 1 columns]


[(0,
  'https://images-na.ssl-images-amazon.com/images/I/51CYReTTtLL._AC_UL200_SR200,200_.jpg'),
 (1,
  'https://images-na.ssl-images-amazon.com/images/I/61N4IJZixJL._AC_UL200_SR200,200_.jpg'),
 (2,
  'https://images-na.ssl-images-amazon.com/images/I/51xx36uTEiL._AC_UL200_SR200,200_.jpg'),
 (3,
  'https://images-na.ssl-images-amazon.com/images/I/81x0XuG1ZEL._AC_UL200_SR200,200_.jpg'),
 (4,
  'https://images-na.ssl-images-amazon.com/images/I/61pK9Pzd8YL._AC_UL200_SR200,200_.jpg'),
 (5,
  'https://images-na.ssl-images-amazon.com/images/I/71tU3GJwnwL._AC_UL200_SR200,200_.jpg'),
 (6,
  'https://images-na.ssl-images-amazon.com/images/I/81%2BP68uE6GL._AC_UL200_SR200,200_.jpg'),
 (7,
  'https://images-na.ssl-images-amazon.com/images/I/81lcSS9uVNL._AC_UL200_SR200,200_.jpg'),
 (8,
  'https://images-na.ssl-images-amazon.com/images/I/71ughJzu-PL._AC_UL200_SR200,200_.jpg'),
 (9,
  'https://images-na.ssl-images-amazon.com/images/I/91eXWdHBMuL._AC_UL200_SR200,200_.jpg'),
 (10,
  'https://images-na.s

In [9]:
## Table2:
print(df_2)

## select from sqlite db
select('select * from image_info')

     Image_name                                          Image_url  Width  \
0             0  https://images-na.ssl-images-amazon.com/images...    200   
1             1  https://images-na.ssl-images-amazon.com/images...    200   
2             2  https://images-na.ssl-images-amazon.com/images...    200   
3             3  https://images-na.ssl-images-amazon.com/images...    200   
4             4  https://images-na.ssl-images-amazon.com/images...    200   
..          ...                                                ...    ...   
113         113  https://images-na.ssl-images-amazon.com/images...     85   
114         114  https://images-na.ssl-images-amazon.com/images...     85   
115         115  https://images-na.ssl-images-amazon.com/images...     85   
116         116  https://images-na.ssl-images-amazon.com/images...     85   
117         117  https://images-na.ssl-images-amazon.com/images...     85   

     Height  Color_channels          Download_datetime  \
0       200      

[(0,
  0,
  'https://images-na.ssl-images-amazon.com/images/I/51CYReTTtLL._AC_UL200_SR200,200_.jpg',
  200,
  200,
  3,
  '2020-10-08 18:50:45.749996',
  '/Users/jamesyang/Desktop/Technical Challenge/download_images/0.jpg'),
 (1,
  1,
  'https://images-na.ssl-images-amazon.com/images/I/61N4IJZixJL._AC_UL200_SR200,200_.jpg',
  200,
  200,
  3,
  '2020-10-08 18:50:45.795479',
  '/Users/jamesyang/Desktop/Technical Challenge/download_images/1.jpg'),
 (2,
  2,
  'https://images-na.ssl-images-amazon.com/images/I/51xx36uTEiL._AC_UL200_SR200,200_.jpg',
  200,
  200,
  3,
  '2020-10-08 18:50:45.839636',
  '/Users/jamesyang/Desktop/Technical Challenge/download_images/2.jpg'),
 (3,
  3,
  'https://images-na.ssl-images-amazon.com/images/I/81x0XuG1ZEL._AC_UL200_SR200,200_.jpg',
  200,
  200,
  3,
  '2020-10-08 18:50:45.887180',
  '/Users/jamesyang/Desktop/Technical Challenge/download_images/3.jpg'),
 (4,
  4,
  'https://images-na.ssl-images-amazon.com/images/I/61pK9Pzd8YL._AC_UL200_SR200,200_.jpg',